### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/PowerAIWithDocker$ jupyter notebook --notebook-dir=/workspace --ip=0.0.0.0 --port=8888 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
https://hub.docker.com/r/continuumio/miniconda3/tags  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
azureml_sdk_version = '1.0.79'
with open('azureml_sdk_version.txt', 'r') as file:
    azureml_sdk_version = (file.read()).rstrip()

azureml_sdk_version

image_version_variable = 'sdk.v'+ azureml_sdk_version 

'1.0.85'

In [3]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [4]:
import dotenv

dotenv.set_key(dotenv.find_dotenv(), 'docker_login', 'georgedockeraccount')
dotenv.set_key(dotenv.find_dotenv(),'image_tag', 'azure_ml_python_sdk_docker_image')
dotenv.set_key(dotenv.find_dotenv(),'image_version', 'sdk.v'+ azureml_sdk_version)

(True, 'docker_login', 'georgedockeraccount')

(True, 'image_tag', 'azure_ml_python_sdk_docker_image')

(True, 'image_version', 'sdk.v1.0.85')

In [5]:
# %%writefile .env

# # Your docker login and image repository name
# docker_login = 'georgedockeraccount'
# image_tag = "azure_ml_python_sdk_docker_image"
# image_version = 'sdk.v'+ azureml_sdk_version   #'sdk.v1.0.81' 

In [6]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [7]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_azure_ml_python_sdk_docker_image_sdk.v1.0.85'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [8]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.7 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain] 
    - azure-cli-core

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [9]:
%%writefile $docker_file_path 

# ARG azureml_python_sdk_version=1.0.79

# FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gpg-agent \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_azure_ml_python_sdk_docker_image_sdk.v1.0.85


In [10]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_azure_ml_python_sdk_docker_image_sdk.v1.0.85'

'georgedockeraccount/azure_ml_python_sdk_docker_image:sdk.v1.0.85'

'georgedockeraccount/azure_ml_python_sdk_docker_imageazcli:sdk.v1.0.85'

'georgedockeraccount/azure_ml_python_sdk_docker_imageazcliazdevops:sdk.v1.0.85'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm0_1 loginvm0_1  334 Jan 22 20:17 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm0_1 loginvm0_1 1881 Jan 22 20:17 dockerfile_azure_ml_python_sdk_docker_image_sdk.v1.0.85


In [11]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
#  ' --build-arg azureml_python_sdk_version='+azureml_sdk_version+' '  + \
build_command

'docker build -t georgedockeraccount/azure_ml_python_sdk_docker_image:sdk.v1.0.85 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_azure_ml_python_sdk_docker_image_sdk.v1.0.85 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

In [12]:
!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 3daec2aa5f38
Removing intermediate container 3daec2aa5f38
 ---> 0621718d49e6
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in f642d3188727
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [173 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7908 kB]
Fetched 8317 kB in 1s (6096 kB/s)
Reading p

Setting up libpython3.7-minimal:amd64 (3.7.3-2) ...
Setting up python3.7-minimal (3.7.3-2) ...
Selecting previously unselected package python3-minimal.
(Reading database ... 12797 files and directories currently installed.)
Preparing to unpack .../python3-minimal_3.7.3-1_amd64.deb ...
Unpacking python3-minimal (3.7.3-1) ...
Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../libmpdec2_2.4.2-2_amd64.deb ...
Unpacking libmpdec2:amd64 (2.4.2-2) ...
Selecting previously unselected package libpython3.7-stdlib:amd64.
Preparing to unpack .../libpython3.7-stdlib_3.7.3-2_amd64.deb ...
Unpacking libpython3.7-stdlib:amd64 (3.7.3-2) ...
Selecting previously unselected package python3.7.
Preparing to unpack .../python3.7_3.7.3-2_amd64.deb ...
Unpacking python3.7 (3.7.3-2) ...
Selecting previously unselected package libpython3-stdlib:amd64.
Preparing to unpack .../libpython3-stdlib_3.7.3-1_amd64.deb ...
Unpacking libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up python3-m

Setting up libmpx2:amd64 (8.3.0-6) ...
Setting up libubsan1:amd64 (8.3.0-6) ...
Setting up libisl19:amd64 (0.20-2) ...
Setting up gpgconf (2.2.12-1+deb10u1) ...
Setting up libcurl4:amd64 (7.64.0-4) ...
Setting up curl (7.64.0-4) ...
Setting up libmpdec2:amd64 (2.4.2-2) ...
Setting up libbinutils:amd64 (2.31.1-16) ...
Setting up cpp-8 (8.3.0-6) ...
Setting up libc-dev-bin (2.28-10) ...
Setting up libcc1-0:amd64 (8.3.0-6) ...
Setting up gpg (2.2.12-1+deb10u1) ...
Setting up liblsan0:amd64 (8.3.0-6) ...
Setting up libitm1:amd64 (8.3.0-6) ...
Setting up binutils-x86-64-linux-gnu (2.31.1-16) ...
Setting up libtsan0:amd64 (8.3.0-6) ...
Setting up pinentry-curses (1.1.0-2) ...
Setting up libpython3.7-stdlib:amd64 (3.7.3-2) ...
Setting up gpg-agent (2.2.12-1+deb10u1) ...
Setting up binutils (2.31.1-16) ...
Setting up libgcc-8-dev:amd64 (8.3.0-6) ...
Setting up cpp (4:8.3.0-1) ...
Setting up libc6-dev:amd64 (2.28-10) ...
Setting up libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up libstdc++-8-de

removing wheel-0.33.6-py37_0
removing blas-1.0-mkl
Removing intermediate container 6a88d43afdad
 ---> dffc2cddf803
Step 8/11 : ENV CONDA_AUTO_UPDATE_CONDA=false
 ---> Running in accb46c38aa8
Removing intermediate container accb46c38aa8
 ---> 0507ae83c4ed
Step 9/11 : ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
 ---> Running in a4577a3d6afa
Removing intermediate container a4577a3d6afa
 ---> 630902758bb2
Step 10/11 : ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
 ---> Running in 5504aa1fb111
Removing intermediate container 5504aa1fb111
 ---> 33bab4a84cdf
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in d613776e705b
Removing intermediate container d613776e705b
 ---> c2a20596a5ec
Successfully built c2a20596a5ec
Successfully tagged georgedockeraccount/azure_ml_python_sdk_docker_image:sdk.v1.0.85
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/17 : MAINTAINER George Iordanes

  1100K .......... .......... .......... .......... .......... 13% 3.90M 0s
  1150K .......... .......... .......... .......... .......... 14%  162M 0s
  1200K .......... .......... .......... .......... .......... 14%  136M 0s
  1250K .......... .......... .......... .......... .......... 15%  233M 0s
  1300K .......... .......... .......... .......... .......... 15%  261M 0s
  1350K .......... .......... .......... .......... .......... 16% 5.76M 0s
  1400K .......... .......... .......... .......... .......... 16%  173M 0s
  1450K .......... .......... .......... .......... .......... 17%  136M 0s
  1500K .......... .......... .......... .......... .......... 18%  202M 0s
  1550K .......... .......... .......... .......... .......... 18%  229M 0s
  1600K .......... .......... .......... .......... .......... 19%  232M 0s
  1650K .......... .......... .......... .......... .......... 19%  427K 1s
  1700K .......... .......... .......... .......... .......... 20%  119M 1s
  1750K ....

  5150K .......... .......... .......... .......... .......... 60%  210M 1s
  5200K .......... .......... .......... .......... .......... 61% 2.59M 1s
  5250K .......... .......... .......... .......... .......... 62%  175M 1s
  5300K .......... .......... .......... .......... .......... 62%  176M 1s
  5350K .......... .......... .......... .......... .......... 63%  205M 1s
  5400K .......... .......... .......... .......... .......... 63%  221M 1s
  5450K .......... .......... .......... .......... .......... 64% 4.43M 1s
  5500K .......... .......... .......... .......... .......... 65%  134M 1s
  5550K .......... .......... .......... .......... .......... 65%  171M 1s
  5600K .......... .......... .......... .......... .......... 66%  222M 1s
  5650K .......... .......... .......... .......... .......... 66%  246M 1s
  5700K .......... .......... .......... .......... .......... 67% 1.41M 1s
  5750K .......... .......... .......... .......... .......... 67%  143M 1s
  5800K ....

Removing intermediate container b2e144e41877
 ---> afc42edb45ff
Step 16/17 : RUN rm -rf /tmp/azcopy
 ---> Running in 1bc8276ad8d1
Removing intermediate container 1bc8276ad8d1
 ---> 66b2131954e7
Step 17/17 : ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH
 ---> Running in 725f1c119ddd
Removing intermediate container 725f1c119ddd
 ---> 89ac4e8aa21f
Successfully built 89ac4e8aa21f
Successfully tagged georgedockeraccount/azure_ml_python_sdk_docker_imageazcli:sdk.v1.0.85
Sending build context to Docker daemon  4.608kB
Step 1/20 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/20 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 0621718d49e6
Step 3/20 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 39fbb2421760

In [13]:
# import docker

# cli = docker.APIClient()
# response=''
# response = [line for line in cli.build(
#     path=working_path, 
#     dockerfile=docker_file_path, 
#     pull=True,
#     rm=True, 
#     quiet=False,
#     nocache=True,
#     tag=aml_docker_image_name
# )]
# response[100:]

# client = docker.from_env()
# client.containers.list()

In [14]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir02']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02'

total 8
-rwxrwxrwx 1 root root 5890 Jan 22 20:10 exported_conda_env.yml


In [15]:
# export the resulted conda env

cli_command_inside_container = ' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; ' + \
    'python -c \'import azureml.core;print(azureml.core.VERSION)\'; ' + \
    'conda env export > /workspace/exported_conda_env.yml"'

cli_command='docker run -it --rm  --name aml-sdk_docker_container ' + \
'-v ' + crt_local_dir + ':/workspace:rw ' + \
aml_docker_image_name + \
cli_command_inside_container

cli_command

if 1:
    ! $cli_command
else:
    client = docker.from_env()
    client.containers.run(aml_docker_image_name, 
                          runtime='',
                          remove=True,
                          volumes={'/workspace': {'bind': '/workspace', 'mode': 'rw'}},
                          working_dir='/',
                          command=cli_command_inside_container)

'docker run -it --rm  --name aml-sdk_docker_container -v /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02:/workspace:rw georgedockeraccount/azure_ml_python_sdk_docker_image:sdk.v1.0.85 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; python -c \'import azureml.core;print(azureml.core.VERSION)\'; conda env export > /workspace/exported_conda_env.yml"'

# conda environments:
#
base                     /opt/conda
aml-sdk-conda-env     *  /opt/conda/envs/aml-sdk-conda-env

/
total 8
-rwxrwxrwx 1 root root 5890 Jan 22 20:10 exported_conda_env.yml
1.0.85


In [16]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [17]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/azure_ml_python_sdk_docker_image:sdk.v1.0.85 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [18]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/azure_ml_python_sdk_docker_image:sdk.v1.0.85 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.85
1.0.85
azure-cli                         2.0.80

command-modules-nspkg              2.0.3
core                              2.0.80
nspkg                              3.0.4
telemetry                          1.0.4

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Jan 13 2020, 05:04:10) 
[GCC 6.3.0 20170516]

Legal docs and information: aka.ms/AzureCliLegal



Your CLI is up-to-date.

Please let us know how we are doing: https://aka.ms/clihats
/bin/bash: azcopy: command not found
1.0.85
azure-cli                         2.0.80

command-modules-nspkg              2.0.3
core                              2.0.80
nspkg                              3.0.4
telemetry                          1.0.4

Extensions:
azure-devops                      0.17.0

Python location '/opt/

#### Do the same in a  notebook

In [19]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [20]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02/test_aml_sdk_docker_image.ipynb


In [21]:
!ls -l $crt_local_dir

total 12
-rwxrwxrwx 1 root       root       5890 Jan 22 20:22 exported_conda_env.yml
-rw-rw-r-- 1 loginvm0_1 loginvm0_1  982 Jan 22 20:22 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [22]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [23]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.85\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [24]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/azure_ml_python_sdk_docker_image]

7e95686c: Preparing 
449e5557: Preparing 
46cae15a: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 


7e95686c: Pushing  2.418GB/2.881GBA

7e95686c: Pushed   2.939GB/2.881GBsdk.v1.0.85: digest: sha256:1c5e8856e56b72476b69279d252190e698efbef57eb2f06ee0ba7a923b0e3e0b size: 1586
The push refers to repository [docker.io/georgedockeraccount/azure_ml_python_sdk_docker_imageazcli]

b7373ddb: Preparing 
72feaf2a: Preparing 
71bd924c: Preparing 
7e95686c: Preparing 
449e5557: Preparing 
46cae15a: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 
71bd924c: Pushed   497.6MB/468.6MBraccount/azure_ml_python_sdk_docker_image Pushing  389.8MB/468.6MBsdk.v1.0.85: digest: sha256:34444de6c4fb09521a1767bec0f2937b7246116e88b2ab91c03099607c2d018d size: 2218
The push refers to repository [docker.io/georgedockeraccount/azure_ml_python_sdk_docker_imageazcliazdevops]

75536524: Preparing 
b7373ddb: Preparing 
72feaf2a: Preparing 
71bd924c: Preparing 
7e95686c: Preparing 
449e5557: Preparing 
46cae15a: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 
5536524: Pushed   17.68MB/16.26MBraccount/azure_ml_python_sdk_docker_imageazcli sdk.v1.0.85

In [25]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 380705 bytes to createAMLSDKDocker.html
